In [1]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "11.0.1" 2018-10-16 LTS; Java(TM) SE Runtime Environment 18.9 (build 11.0.1+13-LTS); Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.1+13-LTS, mixed mode)
  Starting server from /Users/carmen/anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/mf/strl7mm53_dcwkr8qcwz_9fm0000gn/T/tmp5eiez9o1
  JVM stdout: /var/folders/mf/strl7mm53_dcwkr8qcwz_9fm0000gn/T/tmp5eiez9o1/h2o_carmen_started_from_python.out
  JVM stderr: /var/folders/mf/strl7mm53_dcwkr8qcwz_9fm0000gn/T/tmp5eiez9o1/h2o_carmen_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.3
H2O cluster version age:,15 days
H2O cluster name:,H2O_from_python_carmen_cnz3yo
H2O cluster total nodes:,1
H2O cluster free memory:,4 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [2]:
print("Import approved and rejected loan requests...")
loans = h2o.import_file(path = "data/loan.csv")

Import approved and rejected loan requests...
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
# Ensure objetive variable (bad_load) is categorical / factor
loans["bad_loan"] = loans["bad_loan"].asfactor()

In [4]:
# Create training and test partitions
rand  = loans.runif(seed = 1234567)
train = loans[rand <= 0.8]
test = loans[rand > 0.8]

In [5]:
# myY = "bad_loan" objetive 
# myX = Dataset  predictor variables names

myY = "bad_loan"
myX = ["loan_amnt", "longest_credit_length", "revol_util", "emp_length",
       "home_ownership", "annual_inc", "purpose", "addr_state", "dti",
       "delinq_2yrs", "total_acc", "verification_status", "term"]

In [6]:
from h2o.automl import H2OAutoML

In [7]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20,seed=1)
aml.train(x=myX, y=myY, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [8]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,auc,logloss,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20190209_223818,0.684612,0.444771,0.366332,0.373986,0.139866
StackedEnsemble_BestOfFamily_AutoML_20190209_223818,0.683608,0.44507,0.367124,0.374091,0.139944
GBM_grid_1_AutoML_20190209_223818_model_3,0.681532,0.450006,0.369389,0.376051,0.141414
GBM_1_AutoML_20190209_223818,0.681394,0.444588,0.36791,0.374072,0.13993
XGBoost_1_AutoML_20190209_223818,0.681251,0.444438,0.368182,0.373865,0.139775
XGBoost_2_AutoML_20190209_223818,0.67987,0.445039,0.369497,0.374253,0.140065
GBM_2_AutoML_20190209_223818,0.679487,0.445364,0.370572,0.374452,0.140214
GBM_3_AutoML_20190209_223818,0.676891,0.44648,0.370714,0.37496,0.140595
XGBoost_3_AutoML_20190209_223818,0.674515,0.446895,0.373484,0.375076,0.140682
GLM_grid_1_AutoML_20190209_223818_model_1,0.673785,0.447757,0.373899,0.375017,0.140637
